In [1]:
# creating an agentic ai Infowhiz by langchain
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGCHAIN_API_KEY "]= os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")   
os.environ["SERP_API_KEY"]= os.getenv("SERP_API_KEY")

In [3]:
from langchain.chat_models import init_chat_model
#initialising groq model
model = init_chat_model("llama-3.1-8b-instant", model_provider="groq")


/home/suday-nandan-reddy/Desktop/nami/namienv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
#1st retriever tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv)
#2nd retriever tool
from langchain_community.utilities import GoogleScholarAPIWrapper
from langchain_community.tools.google_scholar import GoogleScholarQueryRun

scholar = GoogleScholarAPIWrapper()
scholar_tool = GoogleScholarQueryRun(api_wrapper=scholar)
#3rd retriever tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki)
tools = [arxiv_tool, scholar_tool, wiki_tool]


In [17]:
#for context memory


from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState,StateGraph
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated,TypedDict
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder


from langgraph.prebuilt import create_react_agent



from langchain_core.messages import SystemMessage,trim_messages
from langchain_core.messages.utils import count_tokens_approximately
from langchain_core.messages import HumanMessage,AIMessage
#define a new graph
#language specific output we are creating our own state schema
class State(TypedDict):
    messages:Annotated[Sequence[BaseMessage],add_messages]
    language: str


workflow = StateGraph(state_schema = State)


#model
#prompt


from langchain_core.prompts import PromptTemplate

# template = '''Answer the following questions as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tools}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}'''

# prompt = PromptTemplate.from_template(template)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """You are a helpful research assistant that helps users find information about a given topic in a specified language(if mentioned in the last message query otherwise use english).
            You have access to the following tools: Arxiv, Google Scholar, Wikipedia.Use one of it which is most suitable for the query if You can't find anything useful use another tool.answer for general questions without using tools
"""
         ),
         MessagesPlaceholder(variable_name="messages"),


    ]
)
#def call_model(state:State), we will maintain the context length 
#by trimming the messages based on the model's max token limit


#trimmer
trimmer = trim_messages(
    max_tokens = 1000,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = True,
    start_on = "human"
)

def pre_model_hook(state):
    """
    Runs before each LLM call. Trims the persisted+rehydrated history
    so the live LLM input fits the context window.
    """
    trimmed = trim_messages(
        state["messages"],
        strategy="last", # fast approximate counter
        max_tokens=5000,                           # set for your target model/window
        start_on="human",    
        token_counter = count_tokens_approximately,                       # keep a valid start
        end_on=("human", "tool"),                  # keep a valid end
        include_system=True,                       # preserve system prompt if present
        allow_partial=True,                        # allow partial inclusion of last msg
    )
    # Provide a separate LLM input without mutating persisted memory
    return {"messages": trimmed}
# all can give 500 msgs => if 10 people use this agent we can get 50 msgs per person 








# def call_model(state:State):
#     trimmed_messages = trimmer.invoke(state["messages"])
#     prompt = prompt_template.invoke(
#         {
#             "messages": trimmed_messages,
#             "language": state["language"]
#         }
#     )
#     response = model.invoke(prompt)
#     return {"messages":[response]}


# workflow.add_edge(START,"model")
# workflow.add_node("model",call_model)


memory = MemorySaver()
# app = workflow.compile(checkpointer=memory)
# agent = create_react_agent(
#     app,
#     tools = tools
# )" is this correct instead of giving create_react_agent(model,tools, checkpoint=memory)



agent = create_react_agent(

    model,
    tools,
    checkpointer=memory,
    prompt = prompt_template,
    pre_model_hook=pre_model_hook,
)

In [21]:
config={"configurable": {"thread_id":"abc123"}}
#this messages You will get from databse

query = "ok tell me about ANdhra Pradesh legislative assembly, like who is cm?!!"

input_messages = [HumanMessage(content=query)]

#for rag
#retrieved_snippet = retriever.run(query)

output = agent.invoke(
    {
        "messages": input_messages,
    },
    config,
)
output["messages"]

[HumanMessage(content='Hi i am suday!!', additional_kwargs={}, response_metadata={}, id='a622587f-d90b-41e8-80fd-19d7d2b3bc27'),
 AIMessage(content="It's nice to meet you Suday. What brings you here today? Are you looking for information on a specific topic or do you just want to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 607, 'total_tokens': 640, 'completion_time': 0.062290282, 'prompt_time': 0.037425627, 'queue_time': 0.308898482, 'total_time': 0.099715909}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a8055912-80f0-46c4-8afd-26b484c7b25c-0', usage_metadata={'input_tokens': 607, 'output_tokens': 33, 'total_tokens': 640}),
 HumanMessage(content='what is my name?!!', additional_kwargs={}, response_metadata={}, id='429728ed-6904-424a-88d0-1f35b9f6012d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{